In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_21296\1140549349.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2399 entries, 0 to 2406
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  2399 non-null   object 
 1   date    2399 non-null   object 
 2   ret     2399 non-null   float64
 3   roeq    2399 non-null   float64
 4   mom12m  2399 non-null   float64
dtypes: float64(3), object(2)
memory usage: 112.5+ KB


In [3]:
data.head(3)

,ticker,date,ret,roeq,mom12m
0,JJSF,2021-12,0.161173,0.035539,0.030303
1,ELA,2021-12,0.000000,0.109925,-0.054113
2,PLXS,2021-12,0.139648,0.027229,0.168786


In [4]:
data[["roeq", "mom12m", "ret"]].describe().round(3)

,roeq,mom12m,ret
count,2399.000,2399.000,2399.000
mean,-0.011,0.399,0.023
std,1.899,0.909,0.142
min,-83.912,-0.833,-0.580
25%,-0.007,0.021,-0.023
50%,0.028,0.256,0.036
75%,0.055,0.539,0.085
max,22.041,21.256,3.338


In [5]:
import plotly.express as px
fig = px.scatter(
    data, 
    x="roeq", 
    y="ret", 
    trendline="ols",
    hover_data=dict(
        roeq=False,
        date=False,
        ret=False,
        mom12m=False,
        ticker=True
    ),
)

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=1000,
    height=420,
)

fig.show()

In [6]:
fig = px.scatter(
    data, 
    x="mom12m", 
    y="ret", 
    trendline="ols",
    hover_data=dict(
        roeq=False,
        date=False,
        ret=False,
        mom12m=False,
        ticker=True
    ),
)

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=1000,
    height=420,
)

fig.show()

In [7]:
import pandas as pd

data['roeqpct'] = pd.qcut(
    data.roeq, 
    100, 
    labels=range(1, 101)
)
x = data.groupby("roeqpct").roeq.mean()
y = data.groupby("roeqpct").ret.mean()
df = pd.concat((x,y), axis=1)
df.columns = ["roeq", "ret"]
fig = px.scatter(df, x="roeq", y="ret", trendline="ols")

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=1000,
    height=420,
)

fig.show()

In [8]:
data['roeqpct'] = pd.qcut(
    data.roeq, 
    10, 
    labels=range(1, 11)
)
x = data.groupby("roeqpct").roeq.mean()
y = data.groupby("roeqpct").ret.mean()
df = pd.concat((x,y), axis=1)
df.columns = ["roeq", "ret"]
fig = px.scatter(df, x="roeq", y="ret", trendline="ols")

fig.update_traces(
    marker=dict(
        size=12,
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=1000,
    height=420,
)

fig.show()

In [9]:
data['mom12mpct'] = pd.qcut(
    data.mom12m, 
    100, 
    labels=range(1, 101)
)
x = data.groupby("mom12mpct").mom12m.mean()
y = data.groupby("mom12mpct").ret.mean()
df = pd.concat((x,y), axis=1)
df.columns = ["mom12m", "ret"]
fig = px.scatter(df, x="mom12m", y="ret", trendline="ols")

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=1000,
    height=420,
)

fig.show()

In [10]:
data['mom12mpct'] = pd.qcut(
    data.mom12m, 
    10, 
    labels=range(1, 11)
)
x = data.groupby("mom12mpct").mom12m.mean()
y = data.groupby("mom12mpct").ret.mean()
df = pd.concat((x,y), axis=1)
df.columns = ["mom12m", "ret"]
fig = px.scatter(df, x="mom12m", y="ret", trendline="ols")

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=1000,
    height=420,
)

fig.show()

In [11]:
data["rnk"] = data.ret.rank(pct=True)
fig = px.scatter(
    data, 
    x="roeq", 
    y="rnk", 
    trendline="ols",
    hover_data=dict(
        roeq=False,
        date=False,
        ret=False,
        rnk=False,
        mom12m=False,
        ticker=True
    ),
)

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    width=1000,
    height=420,
)

fig.show()

In [12]:
fig = px.scatter(
    data, 
    x="mom12m", 
    y="rnk", 
    trendline="ols",
    hover_data=dict(
        roeq=False,
        date=False,
        ret=False,
        rnk=False,
        mom12m=False,
        ticker=True
    ),
)

fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    width=1000,
    height=420,
)

fig.show()

In [13]:
data["roeq_rnk"] = data.roeq.rank(pct=True)
fig = px.scatter(
    data, 
    x="roeq_rnk", 
    y="rnk", 
    trendline="ols",
    hover_data=dict(
        roeq=False,
        date=False,
        roeq_rnk=False,
        ret=False,
        rnk=False,
        mom12m=False,
        ticker=True
    ),
)

fig.update_traces(
    marker=dict(
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    width=1000,
    height=420,
)

fig.show()

In [14]:
data["mom12m_rnk"] = data.mom12m.rank(pct=True)
fig = px.scatter(
    data, 
    x="mom12m_rnk", 
    y="rnk", 
    trendline="ols",
    hover_data=dict(
        roeq=False,
        date=False,
        roeq_rnk=False,
        mom12m_rnk=False,
        ret=False,
        rnk=False,
        mom12m=False,
        ticker=True
    ),
)

fig.update_traces(
    marker=dict(
        opacity=0.5
    )
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    width=1000,
    height=420,
)

fig.show()